In [2]:
!pip install pathway bokeh ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 13.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 20

In [3]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()

df = pd.read_csv("dataset.csv")
df


Saving dataset.csv to dataset.csv


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [5]:
df['timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'].astype(str) + ' ' + df['LastUpdatedTime'].astype(str),
    errors='coerce',
    utc=True
)

df = df.dropna(subset=['timestamp'])  # Drop invalid timestamps

# ✅ Assign location_id column directly (instead of renaming)
df['location_id'] = df['SystemCodeNumber'].astype(str)

# ✅ Rename remaining columns
df.rename(columns={
    'Occupancy': 'occupancy',
    'Capacity': 'capacity',
    'QueueLength': 'queue_length',
    'TrafficConditionNearby': 'traffic_level',
    'IsSpecialDay': 'is_special_day',
    'VehicleType': 'vehicle_type'
}, inplace=True)

# ✅ Map traffic conditions
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
df['traffic_level'] = df['traffic_level'].map(traffic_map).fillna(1)

# ✅ Sort by time
df = df.sort_values(by='timestamp')

/tmp/ipython-input-5-431025969.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location_id'] = df['SystemCodeNumber'].astype(str)
/tmp/ipython-input-5-431025969.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={
/tmp/ipython-input-5-431025969.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
# Base demand-based price (Model 2)
def calculate_dynamic_price(
    base_price,
    occupancy,
    capacity,
    queue_length,
    traffic_level,
    is_special_day,
    vehicle_type
):
    weights = {
        'occupancy': 1.0,
        'queue': 0.8,
        'traffic': 0.6,
        'special': 0.5,
        'vehicle': {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
    }

    vehicle_weight = weights['vehicle'].get(str(vehicle_type).lower(), 1.0)

    demand = (
        weights['occupancy'] * (occupancy / max(capacity, 1)) +
        weights['queue'] * queue_length -
        weights['traffic'] * traffic_level +
        weights['special'] * is_special_day +
        vehicle_weight
    )

    demand = max(min(demand, 10), 0)
    normalized_demand = demand / 10
    price = base_price * (1 + normalized_demand)

    return round(min(max(price, base_price * 0.5), base_price * 2.0), 2)


In [10]:
# Haversine formula to compute distance between lat/lon
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # meters
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)

    a = np.sin(dphi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2.0) ** 2
    return 2 * R * np.arcsin(np.sqrt(a))

# Distance matrix between parking lots
location_coords = df.groupby('location_id')[['Latitude', 'Longitude']].first()
location_ids = location_coords.index.tolist()

distances = {
    loc1: {
        loc2: haversine(lat1, lon1, lat2, lon2)
        for loc2, (lat2, lon2) in location_coords.iterrows()
    }
    for loc1, (lat1, lon1) in location_coords.iterrows()
}


In [11]:
def calculate_competitive_price(
    current_loc,
    timestamp,
    base_price,
    occupancy,
    capacity,
    queue_length,
    traffic_level,
    is_special_day,
    vehicle_type,
    current_prices,
    distances,
    radius=500
):
    price = calculate_dynamic_price(
        base_price,
        occupancy,
        capacity,
        queue_length,
        traffic_level,
        is_special_day,
        vehicle_type
    )

    nearby = [loc for loc, dist in distances[current_loc].items() if loc != current_loc and dist <= radius]
    competitor_prices = [current_prices.get(loc, price) for loc in nearby]

    if not competitor_prices:
        return price

    avg_competitor_price = np.mean(competitor_prices)

    # Apply adjustments
    if occupancy >= capacity * 0.95 and price > avg_competitor_price:
        price *= 0.95  # reduce to compete
    elif price < avg_competitor_price:
        price *= 1.05  # raise since others are expensive

    return round(min(max(price, base_price * 0.5), base_price * 2.0), 2)


In [31]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import push_notebook
import pandas as pd

output_notebook()

top_locations = df['location_id'].value_counts().head(3).index.tolist()
sources = {}
colors = ['red', 'green', 'blue']

p = figure(title="Real-Time Dynamic Parking Prices",
           x_axis_type="datetime",
           x_axis_label="Time", y_axis_label="Price ($)",
           width=800, height=400)

for i, loc in enumerate(top_locations):
    sources[loc] = ColumnDataSource(data=dict(x=[], y=[]))
    p.line(x='x', y='y', source=sources[loc], line_width=2,
           legend_label=str(loc), color=colors[i])

p.legend.location = "top_left"
p.y_range.start = 5
p.y_range.end = 25
p.xaxis.major_label_orientation = 0.5

hover = HoverTool(
    tooltips=[("Time", "@x{%F %T}"), ("Price", "@y{$0.00}")],
    formatters={'@x': 'datetime'},
    mode='vline'
)
p.add_tools(hover)

handle = show(p, notebook_handle=True)
current_prices = {}



import asyncio

async def simulate_streaming():
    count = 0
    for _, row in df.iterrows():
        loc = str(row['location_id'])
        if loc not in sources:
            continue

        price = calculate_competitive_price(
            current_loc=loc,
            timestamp=row['timestamp'],
            base_price=10,
            occupancy=row['occupancy'],
            capacity=row['capacity'],
            queue_length=row['queue_length'],
            traffic_level=row['traffic_level'],
            is_special_day=row['is_special_day'],
            vehicle_type=row['vehicle_type'],
            current_prices=current_prices,
            distances=distances
        )

        print(f"[{row['timestamp']}] Location: {loc} → Price: ${price}")

        current_prices[loc] = price
        sources[loc].stream({'x': [row['timestamp']], 'y': [price]}, rollover=300)
        push_notebook(handle=handle)

        count += 1
        if count >= 100:
            break

        await asyncio.sleep(0.2)

await simulate_streaming()




[2016-01-11 08:06:00+00:00] Location: BHMEURBRD01 → Price: $12.25
[2016-01-11 08:06:00+00:00] Location: Others-CCCPS135a → Price: $12.3
[2016-01-11 08:06:00+00:00] Location: BHMNCPHST01 → Price: $12.22
[2016-01-11 08:06:00+00:00] Location: BHMNCPHST01 → Price: $12.72
[2016-01-11 08:26:00+00:00] Location: BHMEURBRD01 → Price: $12.85
[2016-01-11 08:26:00+00:00] Location: Others-CCCPS135a → Price: $14.36
[2016-01-11 08:26:00+00:00] Location: BHMNCPHST01 → Price: $12.27
[2016-01-11 09:00:00+00:00] Location: Others-CCCPS135a → Price: $14.12
[2016-01-11 09:00:00+00:00] Location: BHMNCPHST01 → Price: $12.88
[2016-01-11 09:00:00+00:00] Location: BHMEURBRD01 → Price: $13.39
[2016-01-11 09:26:00+00:00] Location: Others-CCCPS135a → Price: $13.4
[2016-01-11 09:26:00+00:00] Location: BHMNCPHST01 → Price: $12.46
[2016-01-11 09:26:00+00:00] Location: BHMEURBRD01 → Price: $13.48
[2016-01-11 10:00:00+00:00] Location: BHMEURBRD01 → Price: $13.64
[2016-01-11 10:00:00+00:00] Location: Others-CCCPS135a → P

In [18]:
current_prices = {}

def stream_prices():
    count = 0
    for _, row in df.iterrows():
        while not play_toggle.value:
            time.sleep(0.1)

        loc = str(row['location_id'])
        if loc not in sources:
            continue

        try:
            price = calculate_competitive_price(
                current_loc=loc,
                timestamp=row['timestamp'],
                base_price=10,
                occupancy=row['occupancy'],
                capacity=row['capacity'],
                queue_length=row['queue_length'],
                traffic_level=row['traffic_level'],
                is_special_day=row['is_special_day'],
                vehicle_type=row['vehicle_type'],
                current_prices=current_prices,
                distances=distances
            )
            # Corrected indentation for the following lines
            print(f"[{row['timestamp']}] Location: {loc} → Price: ${price}")

            current_prices[loc] = price
            sources[loc].stream({'x': [row['timestamp']], 'y': [price]}, rollover=300)
            push_notebook(handle=handle)
            count += 1

            if count % 50 == 0:
                print(f"{count} rows streamed...")

            time.sleep(speed_slider.value)

        except Exception as e:
            print(f"Error: {e}")

# 🔁 Start in background
threading.Thread(target=stream_prices).start()

[2016-06-11 13:26:00+00:00] Location: BHMNCPHST01 → Price: $17.93
